In [1]:
import logging,qdrant_client
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,Settings,StorageContext
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.qdrant import QdrantVectorStore


/opt/anaconda3/envs/LLAMAINDEX_RAG/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load documents
documents = SimpleDirectoryReader("./doc").load_data()

In [4]:
Settings.llm=Ollama(model="llama3",request_timeout=120.0)

In [5]:
Settings.embed_model=OllamaEmbedding(model_name="snowflake-arctic-embed")

In [6]:
Settings.text_splitter=SentenceSplitter(chunk_size=1024,chunk_overlap=20)

In [7]:
client=qdrant_client.QdrantClient(location=":memory:")
vector_store=QdrantVectorStore(client=client,collection_name="sampledata")

In [8]:
storage_context=StorageContext.from_defaults(vector_store=vector_store)

In [9]:
index = VectorStoreIndex.from_documents(documents=documents,storage_context=storage_context)

In [11]:
index.storage_context.persist(persist_dir="db")

In [12]:
query_engine=index.as_query_engine(response_mode="refine",verbose=True,similarity_top_k=10)

In [26]:
query_str = "Tell me about Ideathon and COMPETITION RULES to participate in ideathon"

In [27]:
system_prompt = (
   "You are an AI assistant specialized in providing information from the uploaded document. "
   "Please ensure that your responses are strictly derived from the content of the document. "
   "If the information is not found in the document, please indicate that explicitly."
)  

In [28]:
query_with_prompt=f"{system_prompt}\nUser query:{query_str}"

In [29]:
query_engine = index.as_query_engine()
response = query_engine.query(query_with_prompt)
display(Markdown(f"{response}"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


According to the provided context information, it appears that "Ideathon" is listed as one of the options under the category "WORKSHOP/SESSION". However, there is no explicit information regarding the competition rules for participating in Ideathon. It's possible that this information may be mentioned elsewhere in the document or presentation, but based on the provided context, I couldn't find any specific details about the competition rules for Ideathon.

In [30]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_with_prompt)
display(Markdown(f"{response}"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


According to the provided context, Ideathon is a pre-event workshop under the ICT Meetup, which involves hands-on problem-solving, empathy-driven ideation, and rapid prototyping techniques. This workshop aims to teach participants to brainstorm, innovate, and transform their ideas into impactful innovations.

To participate in the Ideathon, here are the COMPETITION RULES mentioned in the document:

* Teams should submit ideas based on the theme provided by the organizers.
* Teams can consist of at most four members each.
* The registration will be through our official website.

In [31]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [19]:
hyde_doc

'Here is a passage that answers the question on how to participate in ICT Meetup events:\n\n"Are you interested in exploring the latest developments in Information and Communication Technology (ICT)? Look no further than ICT Meetup, a community-driven platform that brings together professionals, enthusiasts, and students alike to share knowledge, ideas, and experiences. To participate in ICT Meetup events, simply follow these steps: first, create an account on the Meetup website by providing your name, email address, and password. Once you\'ve signed up, search for ICT-related meetups in your area or browse through existing events to find one that piques your interest.\n\nNext, read the event description carefully to understand the topic, agenda, and any specific requirements or expectations from the organizers. If the event aligns with your interests and schedule, click the "Join" button to reserve your spot. You can also RSVP for an event by sending a reply to the organizer\'s email 